In [1]:
from konlpy.tag import Okt
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pprint

## Data import
- Corpus import
- Source 별 Corpus 분류 및 통합 (+유사 콘텐츠 파일 구분)

In [2]:
df1 = pd.read_csv('./data/라그나로크m_INVEN.csv',encoding = 'cp949')
df1['source'] = 'INVEN'
print("INVEN 게시글 dataframe : {}".format(df1.shape))
df1.head()

INVEN 게시글 dataframe : (2794, 5)


date                                               text  \
0  2019-06-13 16:29  https://gamekakaocom/pr/grandchase500Summer?rt...   
1  2019-06-07 14:40                                          JVNQ5UJWW   
2  2019-05-31 20:07  이쁜거만 올릴까 하다가 좀 그래서 재밌는거도 가져왔음 ㅋㅋ 첫번째는 왕좌의 게임 존...   
3  2019-05-29 14:43  MTS44LQGKhttp://romgnjoycom/coupon 에서 쿠폰번호 등록여...   
4  2019-05-29 14:00                                           Pvp하고싶어요   

          title                                           fulltext source  
0  사전 예약 중입니다~!  https://gamekakaocom/pr/grandchase500Summer?rt...  INVEN  
1          6월쿠폰                                      JVNQ5UJWW6월쿠폰  INVEN  
2      다양한 코스프레  이쁜거만 올릴까 하다가 좀 그래서 재밌는거도 가져왔음 ㅋㅋ 첫번째는 왕좌의 게임 존...  INVEN  
3          5월쿠폰  MTS44LQGKhttp://romgnjoycom/coupon 에서 쿠폰번호 등록여...  INVEN  
4       직업추천받아요                                    Pvp하고싶어요직업추천받아요  INVEN

In [3]:
df2 = pd.read_csv('./data/라그나로크m_blogsearch.csv', encoding = 'cp949')
df2['source'] = 'Blogsearch'
df2['fulltext'] = df2['title'] + ' ' + df2['description']
df2.columns = ['title', 'link', ' description', 'date', 'source', 'fulltext']
df2['date'] = pd.to_datetime(df2['date'],format="%Y%m%d")
print("blog검색 결과 dataframe : {}".format(df2.shape))
df2.head()

blog검색 결과 dataframe : (7680, 6)


title  \
0     <b>라그나로크M</b> 공략 환불 순식간에 완료!   
1  <b>라그나로크M</b> 공략 환불 달인이 따로 없네요!   
2      재밌는게임 <b>라그나로크M</b> 직업 뭐 있나   
3   <b>라그나로크M</b> 공략 환불 빠르고 정확하네요!   
4      <b>라그나로크M</b> 1주년 기념 이벤트 소식   

                                                link  \
0  https://blog.naver.com/ssiba9780?Redirect=Log&...   
1  https://blog.naver.com/lpilsoncch?Redirect=Log...   
2  https://blog.naver.com/ppeunimama?Redirect=Log...   
3  https://blog.naver.com/mercialrb?Redirect=Log&...   
4  https://blog.naver.com/musediet?Redirect=Log&l...   

                                         description       date      source  \
0  게임 <b>라그나로크M</b>을 열심히 했었답니다. 최근까지는 재밌게 하다가 아쉽지... 2019-06-21  Blogsearch   
1  업체마다 게임마다 다르겠지만 저 이번에 <b>라그나로크M</b> 환불 받을 때 상담... 2019-06-19  Blogsearch   
2  저는 매년 받는 마카롱 선물과 기다리고 기다렸던 <b>라그나로크m</b> 오픈일로 ... 2018-03-19  Blogsearch   
3  얼마 전까지 제가 하던 <b>라그나로크M</b>이라는 게임에 대해 제가 엄청나게 후... 2019-06-21  Blogsearch   
4  바로 &lt;<b>라그나로크M</b> : 영원한 사랑&gt;인데요. 이 게임은 다른... 2019-03-19  Blogsearch   

                                            fulltext  
0  <b>라그나로크M</b> 공략 환불 순식간에 완료! 게임 <b>라그나로크M</b>을...  
1  <b>라그나로크M</b> 공략 환불 달인이 따로 없네요! 업체마다 게임마다 다르겠지...  
2  재밌는게임 <b>라그나로크M</b> 직업 뭐 있나 저는 매년 받는 마카롱 선물과 기...  
3  <b>라그나로크M</b> 공략 환불 빠르고 정확하네요! 얼마 전까지 제가 하던 <b...  
4  <b>라그나로크M</b> 1주년 기념 이벤트 소식 바로 &lt;<b>라그나로크M</...

In [4]:
df3 = pd.read_csv('./data/라그나로크m_playstore.csv', encoding = 'cp949')
df3['source'] = 'Playstore'
print("플레이스토어 리뷰 dateframe : {}".format(df3.shape))
df3.head()

플레이스토어 리뷰 dateframe : (10148, 5)


date                                           fulltext  \
0  2019-06-11                                                  굿   
1  2019-06-09  재미있어요. 큰 욕심 부리지 않으면 무과금으로도 충분히 즐길만한 게임이에요. 중국 ...   
2  2019-06-08  30~40대의 추억 팔이가 아닌 앞으로도 더더욱 발전하는 라그나로크 게임이 되었스면...   
3  2019-06-08                                            그냥 놀만해요   
4  2019-06-08                                                  굿   

              stars_txt  ratings     source  
0  별표 5개 만점에 5개를 받았습니다.        5  Playstore  
1  별표 5개 만점에 5개를 받았습니다.        5  Playstore  
2  별표 5개 만점에 5개를 받았습니다.        5  Playstore  
3  별표 5개 만점에 4개를 받았습니다.        4  Playstore  
4  별표 5개 만점에 4개를 받았습니다.        4  Playstore

In [5]:
df4 = pd.read_csv('./data/라그나로크m_iOS.csv', encoding = 'cp949')
df4['source'] = 'Appstore'
df4['fulltext'] = df4['review_title'] + df4['review_text']
df4['date'] = pd.to_datetime(df4['date'])
df4['date'] = df4['date'].dt.date
print("iOS 리뷰 dataframe : {}".format(df4.shape))
df4.head()

iOS 리뷰 dataframe : (500, 6)


date          review_title  \
0  2019-06-11        구입목록 삭제좀 부탁드려요   
1  2019-06-06              운영이 어디다?   
2  2019-06-06          다운이 안받아지네요..   
3  2019-06-04                   ㅎ로ㅝ   
4  2019-05-30  12.3.1 업데이트이후 자주팅김현상   

                                         review_text  rating    source  \
0                                     구입목록 삭제좀 부탁드려요       5  Appstore   
1                                          운영이 망친 게임       1  Appstore   
2  아이폰7 용량 큰건데... 다운이 안받아져요ㅡㅡ\n계속 로드중....\n다른 게임은...       3  Appstore   
3                                             ㅠㅠㄹ하ㅏㅐ       5  Appstore   
4                                           문제해결바랍니다       1  Appstore   

                                            fulltext  
0                       구입목록 삭제좀 부탁드려요구입목록 삭제좀 부탁드려요  
1                                  운영이 어디다?운영이 망친 게임  
2  다운이 안받아지네요..아이폰7 용량 큰건데... 다운이 안받아져요ㅡㅡ\n계속 로드중...  
3                                          ㅎ로ㅝㅠㅠㄹ하ㅏㅐ  
4                       12.3.1 업데이트이후 자주팅김현상문제해결바랍니다

In [6]:
df5 = pd.read_csv('./data/라그나로크m_navercafe.csv', encoding = 'cp949')
df5['source'] = 'Navercafe'
df5['date'] = pd.to_datetime(df5['date'])
df5['date'] = df5['date'].dt.date
print("navercafe dataframe : {}".format(df5.shape))
df5.head()

navercafe dataframe : (1500, 3)


date                                           fulltext     source
0  2019-06-12                           한복이 아니라니!!첫번째가...이건 좀...  Navercafe
1  2019-06-12  남녀 구분된 코스튬들 성별별로 저장가능하도록 바꾸주세요무기 코스튬과 킹포링 은혜처럼...  Navercafe
2  2019-06-11                      제련좀어떻게해주시죠...4강에서5는100%아닌가요??  Navercafe
3  2019-06-11  라그m 전용 카카오톡 이모티콘 만들어주세요 ㅎ 길드 수다도 대부분 카카오톡에서 많이...  Navercafe
4  2019-06-04                          60프레임도 공속 정상 적용되게 해주세요지발요  Navercafe

In [7]:
df6 = pd.read_csv('./data/라그나로크m_twit.csv', encoding = 'cp949')
df6['source'] = 'twitter'

def tmp(text):
    text_list = text.split('-')
    return text_list[1].strip()
df6['date'] = df6['date'].apply(tmp)
df6['date'] = pd.to_datetime(df6['date'], format="%Y년 %m월 %d일")

print("Twit dataframe : {}".format(df6.shape))
df6.head()

Twit dataframe : (2180, 3)


fulltext       date   source
0                            라그나로크 M #에피소드 5.0 #미리보기 2019-06-08  twitter
1         #ラグマス 와 이거 라그나로크M이 모바일의 M이 아니라 저런의미엿어? ㅋㅋㅋ 2019-06-05  twitter
2  가만있어보자..12년도니까 무려 7년전이네ㅋㅋㅋ지금까지 촬영한거 옮긴다는게 까먹고 ... 2019-06-04  twitter
3     그라비티, &#039;라그나로크M&#039; 연내 일본 출시 - DAILYFEED  2019-05-27  twitter
4  어제보다 순위가 내린 게임 중 가장 순위가 높은 게임은 「블레이드&소울 레볼루션」(... 2019-05-24  twitter

In [8]:
corpus_df = df1
tmp = [df2,df3,df4,df5,df6]
for df in tmp:
    df1 = df1.loc[:,['date','fulltext','source']].append(df.loc[:,['date','fulltext','source']])

In [9]:
print(df1.columns)
print(df1.shape)

Index(['date', 'fulltext', 'source'], dtype='object')
(24802, 3)


In [10]:
df1['source'].unique()

array(['INVEN', 'Blogsearch', 'Playstore', 'Appstore', 'Navercafe',
       'twitter'], dtype=object)

In [11]:
df1['date'] = pd.to_datetime(df1['date'])

In [12]:
# source별 date 통계
pprint.pprint(df1.groupby(['source'])['date'].min())
pprint.pprint(df1.groupby(['source'])['date'].max())

source
Appstore     2018-04-11 00:00:00
Blogsearch   2017-09-28 00:00:00
INVEN        2018-01-29 17:38:00
Navercafe    2018-05-25 00:00:00
Playstore    2018-03-14 00:00:00
twitter      2017-10-17 00:00:00
Name: date, dtype: datetime64[ns]
source
Appstore     2019-06-11 00:00:00
Blogsearch   2019-06-22 00:00:00
INVEN        2019-06-13 16:29:00
Navercafe    2019-06-12 00:00:00
Playstore    2019-06-11 00:00:00
twitter      2019-06-08 00:00:00
Name: date, dtype: datetime64[ns]


## Data pre-processing

- Text 원문 전처리
- Source별 Corpus 분류
- Stopwrods, 용어 사전 구축
- POS tagging (다른 tagger 적용) 

 + Token Categorization
 + TDM 품사별 분류

In [13]:
df1.head()

date                                           fulltext  \
0 2019-06-13 16:29:00  https://gamekakaocom/pr/grandchase500Summer?rt...   
1 2019-06-07 14:40:00                                      JVNQ5UJWW6월쿠폰   
2 2019-05-31 20:07:00  이쁜거만 올릴까 하다가 좀 그래서 재밌는거도 가져왔음 ㅋㅋ 첫번째는 왕좌의 게임 존...   
3 2019-05-29 14:43:00  MTS44LQGKhttp://romgnjoycom/coupon 에서 쿠폰번호 등록여...   
4 2019-05-29 14:00:00                                    Pvp하고싶어요직업추천받아요   

  source  
0  INVEN  
1  INVEN  
2  INVEN  
3  INVEN  
4  INVEN

### Text 원문 전처리

In [14]:
# 원문 text의 수정
edi_tok = ['잼', '업뎃', '쓰래기','아이탬']
pro_tok = ['재미','업데이트', '쓰레기', '아이템']
df1['fulltext'] = df1['fulltext'].str.replace('전체 리뷰', '')
df1['fulltext'] = df1['fulltext'].str.replace(r'[a-z]+', '')
df1['fulltext'] = df1['fulltext'].str.replace(r'[A-Z]+', '')
df1['fulltext'] = df1['fulltext'].str.replace(r'[-=+,#/<>\;?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》▶●■★◆]', '')
df1['fulltext'] = df1['fulltext'].replace(edi_tok, pro_tok,regex=True) #edt_tok -> pro_tok mapping
df1 = df1[df1['fulltext'].str.len() > 1] #1글자만 있는 row 제거
df1 = df1.drop_duplicates(['fulltext'], keep='first')
doc_to_mat = df1['fulltext'].as_matrix()
doc_to_mat = list(doc_to_mat)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


In [15]:
print(df1.shape)

(14638, 3)


In [16]:
df1.to_csv('corpus_df.csv', index = False)

In [17]:
okt = Okt()
pprint.pprint(okt.tagset)

{'Adjective': '형용사',
 'Adverb': '부사',
 'Alpha': '알파벳',
 'Conjunction': '접속사',
 'Determiner': '관형사',
 'Eomi': '어미',
 'Exclamation': '감탄사',
 'Foreign': '외국어, 한자 및 기타기호',
 'Hashtag': '트위터 해쉬태그',
 'Josa': '조사',
 'KoreanParticle': '(ex: ㅋㅋ)',
 'Noun': '명사',
 'Number': '숫자',
 'PreEomi': '선어말어미',
 'Punctuation': '구두점',
 'ScreenName': '트위터 아이디',
 'Suffix': '접미사',
 'Unknown': '미등록어',
 'Verb': '동사'}


### Stopwrods, 용어 사전 구축
- Tokenizer에 사전 추가

In [18]:
from ckonlpy.tag import Twitter
twitter = Twitter(use_twitter_dictionary=False)
twitter.add_dictionary(['사행성', '핵과금', '무과금', '가챠'], 'Noun') #명사 추가 사전
twitter.add_dictionary(['갑자기','이제','다시', '계속', '역시', '지금', '아주','완전','바로','그냥','계속','정말','아니','자꾸'], 'Adverb') #명사 -> 부사
twitter.add_dictionary(['이건','다른'], 'Determiner') #명사 > 관형사
twitter.add_dictionary(['가요'], 'PreEomi')
twitter.add_dictionary(['재미나다','나다'], 'Verb')
twitter.add_a_template(('Noun', 'Noun', 'Josa'))
stopwords = [
    "이다", "있다", '진짜', '입니다', '게임','하나','모바일',
    '라그나로크', '라그나로크 사랑', '라그나로크 모바일', '모바일 라그나로크','라그', '사랑', '영원한'
    '오늘', '한번','요즘','때문','정도', '같다','어떻다', '아니다', "그렇다", "이렇다", "재미나"]

C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [19]:
def tokenizer(raw, pos=["Noun"], stopword=stopwords): #품사 지정 
    return [
        word for word, tag in twitter.pos(raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

In [20]:
twitter.pos('재미나요')

[('재미나', 'Noun'), ('요', 'Josa')]

### POS tagging
- POS tagging 후 Token Vectorizer 바로 진행 -> Term Document Matrix 구축
- Vectorizer는 카운트 기반과 TF-IDF 기반 두가지를 활용

In [21]:
vectorizer = CountVectorizer(tokenizer=tokenizer, min_df=5, ngram_range=(1,2)) #tokenizer, 최소 출현문서 수, ngram 범위 uni~tri gram (최대출현문서수 추가 가능)
cnt_to_vec = vectorizer.fit_transform(doc_to_mat)
print('fit_transform, (sentence {}, feature {})'.format(cnt_to_vec.shape[0], cnt_to_vec.shape[1]))

fit_transform, (sentence 14638, feature 3834)


In [22]:
features = vectorizer.get_feature_names()
#pprint.pprint(features)

In [23]:
cnt_vec_df = pd.DataFrame(cnt_to_vec.toarray(), columns=features)
cnt_vec_df.head()

가가  가게  가격  가기  가까이  가끔  가나  가능  가능 한일  가능성  ...  흥미  흥미진진  흥행  희귀  희망  흰색  \
0   0   0   0   0    0   0   0   0      0    0  ...   0     0   0   0   0   0   
1   0   0   0   0    0   0   0   0      0    0  ...   0     0   0   0   0   0   
2   0   0   0   0    0   1   0   0      0    0  ...   0     0   0   0   0   0   
3   0   0   0   0    0   0   0   0      0    0  ...   0     0   0   0   0   0   
4   0   0   0   0    0   0   0   0      0    0  ...   0     0   0   0   0   0   

   히드라  히트  힐러  힐링  
0    0   0   0   0  
1    0   0   0   0  
2    0   0   0   0  
3    0   0   0   0  
4    0   0   0   0  

[5 rows x 3834 columns]

In [24]:
tokencnt = cnt_vec_df.sum(axis=0)
tokencnt = tokencnt.sort_values()
print("상위 빈도 token 20개 : \n {}".format(tokencnt[-20:]))

상위 빈도 token 20개 : 
 레벨       423
패치       426
사용       429
사냥       447
운영       450
아이템      456
접속       475
시작       476
카드       495
스킬       514
사전예약     529
이벤트      564
시간       586
사람       586
퀘스트      765
생각       779
업데이트     845
재미       884
추억       885
유저      1002
dtype: int64


In [25]:
# TF_IDF
tfidfvectorizer = TfidfVectorizer(tokenizer=tokenizer, min_df=5,ngram_range=(1,2))    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음)
tfidfvector = tfidfvectorizer.fit_transform(doc_to_mat)
print('fit_transform, (sentence {}, feature {})'.format(tfidfvector.shape[0], tfidfvector.shape[1]))
print(tfidfvector.toarray())
features = tfidfvectorizer.get_feature_names()
#print(features)

fit_transform, (sentence 14638, feature 3834)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [26]:
tfidf_vec_df1 = pd.DataFrame(tfidfvector.toarray(), columns=features)
tfidf_vec_df1.head()

가가   가게   가격   가기  가까이        가끔   가나   가능  가능 한일  가능성  ...   흥미  흥미진진  \
0  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0    0.0  0.0  ...  0.0   0.0   
1  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0    0.0  0.0  ...  0.0   0.0   
2  0.0  0.0  0.0  0.0  0.0  0.193767  0.0  0.0    0.0  0.0  ...  0.0   0.0   
3  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0    0.0  0.0  ...  0.0   0.0   
4  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0    0.0  0.0  ...  0.0   0.0   

    흥행   희귀   희망   흰색  히드라   히트   힐러   힐링  
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 3834 columns]

In [27]:
tfidf_vec_df1.shape

(14638, 3834)

In [28]:
cnt_vec_df['date'] = list(df1['date'])
cnt_vec_df['source'] = list(df1['source'])

tfidf_vec_df1['date'] = list(df1['date'])
tfidf_vec_df1['source'] = list(df1['source'])

In [29]:
cnt_vec_df.to_csv('noun_cnt_tdm.csv', index = False)
tfidf_vec_df1.to_csv('noun_tfidf_tdm.csv',index=False)

In [30]:
max_tfidf = tfidf_vec_df1.iloc[:,:-2].sum(axis=0).sort_values()
print("tf-idf 상위 20 token: \n{}".format(max_tfidf[-20:]))

tf-idf 상위 20 token: 
예전       94.257797
사람       97.689949
쿠폰       97.866308
운영       99.771483
처음      103.034693
이벤트     104.315792
향수      111.175180
시작      111.644479
최고      114.987146
유저      121.501403
접속      123.636308
윤아      125.458008
시간      141.410223
퀘스트     149.968110
사전예약    160.763924
생각      160.878138
옛날      162.309939
업데이트    180.609936
추억      430.096407
재미      463.994153
dtype: float64


In [31]:
def token_doc(keyword = str(input())):
    srch=[t for t in tokenizer(keyword) if t in features]
    print("keyword token" , srch)
    srch_dtm_tfidf = np.asarray(tfidfvector.toarray())[:, [
        # vectorize.vocabulary_.get 는 특정 feature 가 dtm 에서 가지고 있는 index값을 리턴한다
        tfidfvectorizer.vocabulary_.get(i) for i in srch]]
    temp_df = pd.DataFrame(srch_dtm_tfidf, columns=srch)
    temp_df = temp_df[(temp_df > 0.0).all(axis=1)] #하나의 token tf-idf값이 0이 있으면 제외
    search_score_tfidf = temp_df.sum(axis=1)
    print('related documents : {}'.format(len(search_score_tfidf)))
    for i,j in zip(search_score_tfidf.sort_values(ascending=False).index, search_score_tfidf.sort_values(ascending=False).values):
        print('{} / tf-idf score : {}'.format(doc_to_mat[i], j))

token_doc()

 사람


keyword token ['사람']
related documents : 430
내가 좋아하는 사람 너무 좋아 진짜 너무 좋아 / tf-idf score : 1.0
신섭안나와요 신섭기다리는사람 많은데 / tf-idf score : 1.0
전에 하던사람이라 그런지 몰라도 디지게재밌네여 / tf-idf score : 1.0
나랑 14일부터 라그나로크할사람 / tf-idf score : 1.0
정말 많은 사람이 있습니다 / tf-idf score : 1.0
그러나 사람이없다 / tf-idf score : 1.0
사람 좀 없겠다 예상한서버로 변경했는데도 사람 많이 보이네요 ㅎㅎ북적북적ㅎㅎㅎㅎ사람들 많네요 ㅎㅎㅎ / tf-idf score : 0.7793173141371019
좋은 사람들과 함께하는 행복한 시간을 보내는 / tf-idf score : 0.7170385221850986
다들 냉방병 조심하세요다들 장난아니네요ㅠ제 주변사람들도 냉방병때문에 아픈 사람들 진짜 많음라그엠할때 환기 하면서 하세요급해도ㅎㅎ다들 냉방병조심하세요ㅎㅎ / tf-idf score : 0.6699218893502208
ㅈㄱㄴ초반그닥 평이 좋지 않았던거 같은데하는 사람 꽤 있는 편인가요 / tf-idf score : 0.6103618337177737
진짜 답답하네실수한 사람이나 그거 알면서도 제니 사용한 사람이나멍청이가 아니고서야 그거 사용하면 날리난다는걸 모를까  그리고 먼 말을 해야될거 아니야 소통을 해야지시간 앞부분만 몰래 와서 22 로 바꾸고 가면 어떻사람들이 좋아해  머 할사람들은 하겠지 머 이런 생각 가지고운영 하는거지 사람들 호구로 보는건 나라 나 기업이나똑같네 / tf-idf score : 0.5945150747654568
140대존에 사람들이 많이 줄어있는게 눈에띄네요7섭이라 그런가1달전까지만해도 해저동굴도 사람들 미어터졌었는데오픈하고 2달다되가니 사람좀 줄은듯한느낌 / tf-idf score : 0.5927842818573403
워  놋 8 쓰는 사람인데요 들어갈때마다 로딩 

### 형용사 tdm 구축 

In [32]:
def tokenizer2(raw, pos=["Adjective"], stopword=stopwords): #품사 지정 : 형용사
    return [
        word for word, tag in twitter.pos(raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

In [33]:
vectorizer = CountVectorizer(tokenizer=tokenizer2, min_df=5, ngram_range=(1,1)) #tokenizer, 최소 출현문서 수, ngram 범위 uni~tri gram (최대출현문서수 추가 가능)
cnt_to_vec = vectorizer.fit_transform(doc_to_mat)
print('fit_transform, (sentence {}, feature {})'.format(cnt_to_vec.shape[0], cnt_to_vec.shape[1]))

fit_transform, (sentence 14638, feature 265)


In [34]:
features = vectorizer.get_feature_names()
#pprint.pprint(features)

In [35]:
cnt_vec_df = pd.DataFrame(cnt_to_vec.toarray(), columns=features)
cnt_vec_df.head()

가깝다  가능하다  가득하다  가볍다  간단하다  간절하다  간편하다  강력하다  강하다  강화하다  ...  확실하다  활발하다  \
0    0     0     0    0     0     0     0     0    0     0  ...     0     0   
1    0     0     0    0     0     0     0     0    0     0  ...     0     0   
2    0     0     0    0     0     0     0     0    0     0  ...     0     0   
3    0     0     0    0     0     0     0     0    0     0  ...     0     0   
4    0     0     0    0     0     0     0     0    0     0  ...     0     0   

   황당하다  훌륭하다  흔하다  흡사하다  흥미롭다  희다  힘드다  힘들다  
0     0     0    0     0     0   0    0    0  
1     0     0    0     0     0   0    0    0  
2     0     0    0     0     0   0    0    0  
3     0     0    0     0     0   0    0    0  
4     0     0    0     0     0   0    0    0  

[5 rows x 265 columns]

In [36]:
tokencnt = cnt_vec_df.sum(axis=0)
tokencnt = tokencnt.sort_values()
print("상위 빈도 token 20개 : \n {}".format(tokencnt[-20:]))

상위 빈도 token 20개 : 
 아기자기하다     145
높다         147
심하다        150
아쉽다        152
어렵다        156
귀엽다        160
힘들다        171
괜찮다        178
새롭다        192
필요하다       215
불편하다       240
가능하다       264
영원하다       328
부탁드리다      421
많다         682
재미있다       818
안되다       1122
재밌다       1139
없다        1814
좋다        1913
dtype: int64


In [37]:
# TF_IDF
tfidfvectorizer = TfidfVectorizer(tokenizer=tokenizer2, min_df=5,ngram_range=(1,1))    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음)
tfidfvector = tfidfvectorizer.fit_transform(doc_to_mat)
print('fit_transform, (sentence {}, feature {})'.format(tfidfvector.shape[0], tfidfvector.shape[1]))
print(tfidfvector.toarray())
features = tfidfvectorizer.get_feature_names()
#print(features)

fit_transform, (sentence 14638, feature 265)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [38]:
tfidf_vec_df2 = pd.DataFrame(tfidfvector.toarray(), columns=features)
tfidf_vec_df2.head()

가깝다  가능하다  가득하다  가볍다  간단하다  간절하다  간편하다  강력하다  강하다  강화하다  ...  확실하다  활발하다  \
0  0.0   0.0   0.0  0.0   0.0   0.0   0.0   0.0  0.0   0.0  ...   0.0   0.0   
1  0.0   0.0   0.0  0.0   0.0   0.0   0.0   0.0  0.0   0.0  ...   0.0   0.0   
2  0.0   0.0   0.0  0.0   0.0   0.0   0.0   0.0  0.0   0.0  ...   0.0   0.0   
3  0.0   0.0   0.0  0.0   0.0   0.0   0.0   0.0  0.0   0.0  ...   0.0   0.0   
4  0.0   0.0   0.0  0.0   0.0   0.0   0.0   0.0  0.0   0.0  ...   0.0   0.0   

   황당하다  훌륭하다  흔하다  흡사하다  흥미롭다   희다  힘드다  힘들다  
0   0.0   0.0  0.0   0.0   0.0  0.0  0.0  0.0  
1   0.0   0.0  0.0   0.0   0.0  0.0  0.0  0.0  
2   0.0   0.0  0.0   0.0   0.0  0.0  0.0  0.0  
3   0.0   0.0  0.0   0.0   0.0  0.0  0.0  0.0  
4   0.0   0.0  0.0   0.0   0.0  0.0  0.0  0.0  

[5 rows x 265 columns]

In [39]:
tfidf_vec_df2.shape

(14638, 265)

In [40]:
cnt_vec_df['date'] = list(df1['date'])
cnt_vec_df['source'] = list(df1['source'])

tfidf_vec_df2['date'] = list(df1['date'])
tfidf_vec_df2['source'] = list(df1['source'])

In [41]:
cnt_vec_df.to_csv('adject_cnt_tdm.csv', index = False)
tfidf_vec_df2.to_csv('adject_tfidf_tdm.csv',index=False)

In [43]:
max_tfidf = tfidf_vec_df2.iloc[:,:-2].sum(axis=0).sort_values()
print("tf-idf 상위 20 token: \n{}".format(max_tfidf[-20:]))

tf-idf 상위 20 token: 
기대하다        92.407283
심하다         97.841928
검다         100.490359
어렵다        100.611649
아기자기하다     105.087318
필요하다       107.310460
귀엽다        107.517032
힘들다        112.605228
괜찮다        114.935753
새롭다        117.105188
가능하다       135.260046
불편하다       139.075020
영원하다       257.651222
부탁드리다      260.660695
많다         319.323174
재미있다       610.450243
안되다        663.805319
재밌다        827.529984
없다         847.924932
좋다        1063.653048
dtype: float64


In [44]:
srch=[t for t in tokenizer2("힘들다") if t in features]
print("keyword token" , srch)
if len(srch) >= 1:
    srch_dtm_tfidf = np.asarray(tfidfvector.toarray())[:, [
        # vectorize.vocabulary_.get 는 특정 feature 가 dtm 에서 가지고 있는 index값을 리턴한다
        tfidfvectorizer.vocabulary_.get(i) for i in srch]]
    temp_df = pd.DataFrame(srch_dtm_tfidf, columns=srch)
    temp_df = temp_df[(temp_df != 0).all(axis=1)] #하나의 token tf-idf값이 0이 있으면 제외
    search_score_tfidf = temp_df.sum(axis=1)
    print('related documents : {}'.format(len(search_score_tfidf)))
    for i,j in zip(search_score_tfidf.sort_values(ascending=False).index, search_score_tfidf.sort_values(ascending=False).values):
        print('{} / tf-idf score : {}'.format(doc_to_mat[i], j))

keyword token ['힘들다']
related documents : 163
저 지스타 왔어요 헤헤이 부스 저 부스 돌아다니고 있는데 발견한 포링 인형뽑기아 졸귀탱 ㅠㅠㅠㅠ 큰게 탐나는데 너무 커서 뽑기는 힘들듯 헤헿지스타 그라비티 라그나로크 라그나로크 라그 포링 / tf-idf score : 1.0
초반에 과금 안하면 많이 힘듬 / tf-idf score : 1.0
컨트롤이 조금힘들어요 / tf-idf score : 1.0
옛날생각나네요 ㅋㅋ 카드먹기힘들당 / tf-idf score : 1.0
힘들어 / tf-idf score : 1.0
컨트롤이 좀힘듬 / tf-idf score : 1.0
으시발 힘들다라그나로크  / tf-idf score : 1.0
자주 팅 갬 하기 힘듬 / tf-idf score : 1.0
게임성 괜챦음 컨트롤은 오토아니면 힘듬 렙업 너무빡쎔 / tf-idf score : 1.0
아이템 드랍율 좀 높여주세요 재료는 많이 모으라고 해놓고 너무 모으기가 힘들어요 / tf-idf score : 1.0
오늘 다운 힘들게 다 받았는데 계속 튕김아이폰 6쓰고있는 군인 인데요 
힘들게 다운받고 열심히 해야지 했는데  
로그인하고 캐릭터 들어가면 계속 튕겨요 캐릭터를 움직여 보지도 못하고 계속 튕기네요 어떻게좀 해주세요 / tf-idf score : 1.0
다운로드로 시간다 보냄 진짜 힘듬 / tf-idf score : 1.0
거참 채널이동만 수공으로 알아서됐으면 워프관리인 찾고 어느세월에 네임드는 보이지도않고 잡기도힘들고 / tf-idf score : 1.0
나혼자한다 라는 이런것을 좀더 밀어줘요 아싸는 힘들어요 ㅠㅠ / tf-idf score : 1.0
퀘스트 부족 및 재료 찾기 힘듬 / tf-idf score : 1.0
클릭 보정이 안돼고 퀘스트보기도 힘듦 / tf-idf score : 1.0
생각보다 꿀재미맵이 너무 크긴 함걸어다니기 힘들 / tf-idf score : 1.0
퀘가 너무 힘듬 ㅠㅠ / tf-idf score : 1.0
팅겨

## Word2vec

In [45]:
df1.head()

date                                           fulltext  \
0 2019-06-13 16:29:00              500_02001999574350488884640사전 예약 중입니다   
1 2019-06-07 14:40:00                                              56월쿠폰   
2 2019-05-31 20:07:00  이쁜거만 올릴까 하다가 좀 그래서 재밌는거도 가져왔음 ㅋㅋ 첫번째는 왕좌의 게임 존...   
3 2019-05-29 14:43:00                    44 에서 쿠폰번호 등록여태 잊어먹었네요 잘쓰석요5월쿠폰   
4 2019-05-29 14:00:00                                       하고싶어요직업추천받아요   

  source  
0  INVEN  
1  INVEN  
2  INVEN  
3  INVEN  
4  INVEN

In [46]:
sentence_token = list(df1['fulltext'].apply(tokenizer))

In [47]:
from gensim.models import Word2Vec
embedding_model = Word2Vec(sentence_token, size=128, window = 2, min_count=10, workers=4, iter=100, sg=1)

## Word Similarity using word2vec

In [48]:
# 특정 token(위의 case는 명사)과 가장 유사도가 큰 토큰 50개를 도출
pprint.pprint(embedding_model.most_similar(positive=["추억"], topn=50))

[('감성', 0.5052341818809509),
 ('학생', 0.47631943225860596),
 ('고딩', 0.47605133056640625),
 ('팔이', 0.47478872537612915),
 ('베리', 0.4729290008544922),
 ('장수', 0.46984803676605225),
 ('명작', 0.46047884225845337),
 ('재미', 0.4595166742801666),
 ('미남', 0.4480975270271301),
 ('새록새록', 0.445407897233963),
 ('온라인', 0.44353336095809937),
 ('게이머', 0.44285011291503906),
 ('사운드', 0.43955177068710327),
 ('대작', 0.43516698479652405),
 ('옛날', 0.42779263854026794),
 ('중국산', 0.42543166875839233),
 ('햇던', 0.42472517490386963),
 ('기억', 0.41849392652511597),
 ('재현', 0.41814982891082764),
 ('대감', 0.4113041162490845),
 ('플랫폼', 0.4097617268562317),
 ('핵꿀', 0.409640371799469),
 ('대중', 0.40545380115509033),
 ('몰입', 0.40299302339553833),
 ('강추', 0.4015967845916748),
 ('상상', 0.40072739124298096),
 ('평점', 0.3962063789367676),
 ('뿜뿜', 0.3939412832260132),
 ('굿굿', 0.3921773135662079),
 ('인생게임', 0.3917895257472992),
 ('온라인게임', 0.38960182666778564),
 ('역대', 0.3866090774536133),
 ('아기자기', 0.38519883155822754),
 ('융시', 0.38

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


In [49]:
import gensim
word_vectors = embedding_model.wv
vocabs = word_vectors.vocab.keys()
print(len(vocabs))

1787


In [50]:
index = gensim.similarities.MatrixSimilarity(gensim.matutils.Dense2Corpus(embedding_model.wv.syn0.T))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


In [51]:
embedding_model.wv.init_sims()
similarity_matrix = np.dot(embedding_model.wv.syn0norm, embedding_model.wv.syn0norm.T) # word2vec similarity matrix

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.vectors_norm instead).
  


In [52]:
similarity_matrix

array([[ 1.        ,  0.2715334 ,  0.15025677, ...,  0.30359304,
         0.20817935,  0.27837133],
       [ 0.2715334 ,  1.0000001 ,  0.45951667, ...,  0.17356966,
         0.18534477, -0.06608984],
       [ 0.15025677,  0.45951667,  1.        , ...,  0.06394241,
         0.13078916,  0.00910688],
       ...,
       [ 0.30359304,  0.17356966,  0.06394241, ...,  1.0000001 ,
         0.54325527,  0.5116782 ],
       [ 0.20817935,  0.18534477,  0.13078916, ...,  0.54325527,
         1.0000001 ,  0.39832368],
       [ 0.27837133, -0.06608984,  0.00910688, ...,  0.5116782 ,
         0.39832368,  0.99999994]], dtype=float32)

In [53]:
def term_freq(doc):
    doc_dict = {}
    for term in list(vocabs):
        doc_dict[term] = doc.count(term)
    return doc_dict

tdm_list = list(df1['fulltext'].apply(term_freq))

In [54]:
tdm = pd.DataFrame.from_dict(tdm_list).fillna(0)

In [55]:
tdm = tdm.loc[:,list(vocabs)]

In [56]:
tdm.head()

사전  예약  세번  각성  코스프레  가끔  재미  쿠폰  번호  등록  ...  탐욕  농축  디토  세크리파이스  오른  코리아  \
0   1   1   0   0     0   0   0   0   0   0  ...   0   0   0       0   0    0   
1   0   0   0   0     0   0   0   1   0   0  ...   0   0   0       0   0    0   
2   0   0   1   2     4   1   1   0   0   0  ...   0   0   0       0   0    0   
3   0   0   0   0     0   0   0   2   1   1  ...   0   0   0       0   0    0   
4   0   0   0   0     0   0   0   0   0   0  ...   0   0   0       0   0    0   

   기자  강준  브레이크뉴스  융시  
0   0   0       0   0  
1   0   0       0   0  
2   0   0       0   0  
3   0   0       0   0  
4   0   0       0   0  

[5 rows x 1787 columns]

In [57]:
df1['fulltext']

0                   500_02001999574350488884640사전 예약 중입니다
1                                                   56월쿠폰
2       이쁜거만 올릴까 하다가 좀 그래서 재밌는거도 가져왔음 ㅋㅋ 첫번째는 왕좌의 게임 존...
3                         44 에서 쿠폰번호 등록여태 잊어먹었네요 잘쓰석요5월쿠폰
4                                            하고싶어요직업추천받아요
5       아니 과금 ㅈ망겜이냐걍 정상적으로 평타만 치려해도돈 조지게 필하누나같은 편돌이는 뭐...
6       인벤은 포켓몬 도감같은 느낌으로 와서 조합재료 같은거 구경하고 가는 장소임질문글또한...
7                                           평균 렙이 몇정도 되나요
8                                    캐릭머로하면 좋을까요과금하려고 하는데
9                                         라그 할때 꿀팁같은거 있나요
10                                            333직업추천해주세요
11      업데이트이 그쯤에서 끊겻으니 100렙이상 질문사항은공식 카페로 가십쇼 라그인벤은 정...
12      무기 커즈드 라이어  갑옷 레인져 의상 신발 고급 야성 부츠  악세 레드마노 헤어핀...
13                                              마격 부탁 드려요
14                                          봤어요ㅋ 잘쓰셈봐염ㅋ1명
15                                          할만한가요라그엠 어떤가요
16      모바일 게임 지금 딱 2개하고있는데라그이랑 트라하라그은 진짜 출퇴근길에 멍때리면서가...
17      첨에 어플 

In [58]:
len(tdm.columns)

1787

In [59]:
similarity_matrix_df = pd.DataFrame(similarity_matrix, index=list(vocabs), columns = list(vocabs))

In [60]:
similarity_matrix_df.shape

(1787, 1787)

In [61]:
tdm.shape

(14638, 1787)

In [62]:
word2vec_doc_score = np.dot(similarity_matrix_df.values, tdm.values.T)

In [63]:
similarity_matrix_df.head()

사전        예약        세번        각성      코스프레        가끔        재미  \
사전    1.000000  0.271533  0.150257  0.189174  0.263659  0.071296  0.137191   
예약    0.271533  1.000000  0.459517  0.177958  0.204578  0.107293  0.198521   
세번    0.150257  0.459517  1.000000  0.215706  0.138731  0.243527  0.238260   
각성    0.189174  0.177958  0.215706  1.000000  0.108582  0.187237  0.131984   
코스프레  0.263659  0.204578  0.138731  0.108582  1.000000  0.190724  0.143679   

            쿠폰        번호        등록  ...        탐욕        농축        디토  \
사전    0.472016  0.120737  0.085667  ...  0.196208  0.189799  0.075606   
예약    0.170815  0.145656  0.251565  ...  0.154199  0.197134  0.409640   
세번    0.166301  0.082977  0.225512  ...  0.061778  0.142936  0.544737   
각성    0.073674  0.269835  0.235871  ...  0.303257  0.114653  0.230001   
코스프레  0.243266  0.069861  0.100658  ...  0.237147  0.261043  0.170446   

        세크리파이스        오른       코리아        기자        강준    브레이크뉴스        융시  
사전    0.413490  0.303084  0.255432  0.160421  0.303593  0.208179  0.278371  
예약    0.230084  0.235742  0.209406  0.158817  0.173570  0.185345 -0.066090  
세번    0.082329  0.192252  0.149259  0.147108  0.063942  0.130789  0.009107  
각성    0.204124  0.101582  0.340738  0.244326  0.217956  0.200763  0.274186  
코스프레  0.211437  0.267021  0.195781  0.145275  0.233086  0.242940  0.267685  

[5 rows x 1787 columns]

In [64]:
word2vec_doc_score_df = pd.DataFrame(word2vec_doc_score.T, columns = list(vocabs))

In [65]:
word2vec_doc_score_df.head()

사전        예약        세번        각성      코스프레        가끔        재미  \
0  1.271533  1.271534  0.609773  0.367131  0.468237  0.178589  0.335712   
1  0.472016  0.170815  0.166301  0.073674  0.243266  0.095118  0.140565   
2  2.273687  3.356238  3.624529  2.993922  5.897945  3.925517  3.324992   
3  1.229131  0.817745  0.692831  0.804561  0.721252  0.782314  0.602609   
4  0.256635  0.400672  0.326244  0.243901  0.166766  0.280087  0.365981   

         쿠폰        번호        등록  ...        탐욕        농축        디토    세크리파이스  \
0  0.642831  0.266393  0.337231  ...  0.350408  0.386933  0.485246  0.643574   
1  1.000000  0.179912  0.142114  ...  0.129129  0.166441  0.124875  0.324377   
2  2.242500  2.465643  1.951220  ...  2.472113  3.413057  3.670327  2.360988   
3  2.402213  1.683617  1.678746  ...  0.711821  0.638210  0.755036  0.847149   
4  0.232313  0.369115  0.429411  ...  0.205674  0.314775  0.466988  0.111291   

         오른       코리아        기자        강준    브레이크뉴스        융시  
0  0.538826  0.464838  0.319238  0.477163  0.393524  0.212281  
1  0.198740  0.171429  0.164931  0.243950  0.159794  0.107615  
2  3.126971  2.777795  2.300831  2.838229  3.320447  3.350693  
3  0.726885  0.627427  1.039196  1.230319  0.981562  0.628598  
4  0.300862  0.375809  0.234343  0.286199  0.369984  0.217408  

[5 rows x 1787 columns]

In [66]:
word2vec_doc_score_df.shape

(14638, 1787)

In [67]:
word2vec_doc_score_df.columns

Index(['사전', '예약', '세번', '각성', '코스프레', '가끔', '재미', '쿠폰', '번호', '등록',
       ...
       '탐욕', '농축', '디토', '세크리파이스', '오른', '코리아', '기자', '강준', '브레이크뉴스', '융시'],
      dtype='object', length=1787)

In [68]:
# keyword 별 가장 높은 Score를 가지는 문서 top N개를 도출하는 함수
def extract_docs(keyword, n):
    tmp_df = word2vec_doc_score_df[keyword].sort_values(ascending = False)
    get_nindex = tmp_df.index[:n]
    for text in np.ndarray.tolist(df1.iloc[get_nindex, 1].values):
        print(text, '\n')

extract_docs('향수', 5)

라그나로크 인벤오픈 축하드립니다라그나로크 은 얼마나 유저의견을 반영할지 혹은제로처럼 본서버 유저의 아이디어만 빼먹을지 궁금하네요4년동안 그라비티 11 문의로 건의 해도 무시라그나로크 인벤 그라비티에 바란다에 2년동안 건의 해도 무시라그나로크 제로는 인벤 만들기도 싫어서라그나로크 인벤 꼽사리 껴두게 해두신분들이라그나로크은 바로 인벤을 만들어주시는군요무시 그만하시고 오픈전에 라그나로크 본서버 의견좀 제대로 인식좀 해주셨으면 했습니다본서버 운영인원이랑 제로서버운영인원이 따로 있다는 거짓말은 진작에 들통났는데이젠 제로서버 인원조차 라그 신경쓴다고빠져나갈까봐 걱정부터 되는군요본서버의 아래의 문제들 언제쯤이나해결해볼 시늉이나마해주실건가요읽기는 했습니까문제는 인지하고 있나요11 문의건 그라비티에 바란다 이건확인했습니다 따위의 말만 하지말고언제까지 어떻게 할건지 명확한 답변좀 주셨으면 합니다왜 유저들을 그렇게 무시하기만 하시는 지요어차피 라그제로다 이다 이런거 신경쓰느라수년간은 본서버를 신경써줄것 같지도 않습니다만광고비 조금이라도 아껴서 인력보충좀 하고유지보수만이라도 좀 해주셨으면 합니다허구헌날 서버 터지는거도 지겹습니다정말 본서버는 닫을 준비라도 하시는겁니까클래식 서버는 안연다고 하고 제로를 열었던 통수처럼본서버는 닫지 않는다라고 말하고 서비스를 종료합니다그동안 사랑해 주셔서 감사합니다라는 통수가 나올꺼 같은 기분이 드는군요아래는 현재 24일간 매일 적고 있는라그나로크 본서버의 문제와 건의 사항들입니다여기에 대한 답변 여기서라도 주시면 좋겠네요확인했습니다 라는 말장난만 하지 말아주세요눕눕24일차추가팔척뛰기님의견1191 에피소드 171 개발 에피소드 업데이트가 162 테라 글로리아 이후로 무려 1년 10개월 2개월 더 있으면 2년이 넘어가는 세월동안 한번도 없다2 일루전 컨텐츠 나머지 다 내놓기 작년 5월 중순 일루전 구양궁 이후론 내놓은 것이 전혀 없다 그래 뭐 3월달엔 테디베어 나오니까 그건 빼고 혹시나 까먹었을까봐 목록 나열해 드림 일루전 코모도 일루전 어비스 레이크 